In [1]:
from joblib import load
svm_best_model = load("Models/svm_full.joblib")

First split the data

In [2]:
import pandas as pd

# Comments File
comments = 'Data/combined_comments.csv'
comm = pd.read_csv(comments)
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment")
unknown = (comm.sectionName == "Unknown") & (comm.commentType == "comment")
# Data that is known section, for training
data = comm.commentBody[filt]
sections = comm.sectionName[filt]
# Data that is unknown, for prediction
data_unknown = comm.commentBody[unknown]
sections_unknown = comm.sectionName[unknown]

In [3]:
sections_pd = pd.DataFrame(sections)
sections_pd.columns = ['Training_count']
rows_training_count = sections_pd.shape[0]
training_spread = sections_pd['Training_count'].value_counts()

def get_percentage_training(num):
    return float(num)/rows_training_count

In [4]:
# This is the report for training data
training_spread = pd.DataFrame(training_spread)

In [5]:
training_spread['Percent_training'] = training_spread.Training_count.apply(get_percentage_training)
training_spread['Index'] = training_spread.index

In [6]:
# This is the prediction for data that is in unknown section
results = svm_best_model.predict(data_unknown)

In [7]:
output = pd.concat([data_unknown.reset_index(drop=True),pd.DataFrame(results)], axis=1)
output.columns = ['comments','labels']

In [8]:
# Export the result to csv
output.to_csv('comment_results.csv', sep=',')

In [9]:
# The result of section assignment for unknown labels
output['labels'].value_counts()

Politics                      408967
Sunday Review                 236518
Europe                         14010
Family                         12208
Middle East                     9913
Economy                         7701
Live                            6560
Asia Pacific                    6220
Media                           5939
Television                      4505
Art & Design                    3259
Energy & Environment            2416
DealBook                        2214
Move                            1924
Eat                             1456
Pro Football                    1393
Americas                        1239
Olympics                        1225
Baseball                        1075
Music                            863
Canada                           754
Wine, Beer & Cocktails           700
College Basketball               364
Pro Basketball                   308
Mind                             302
Book Review                      301
Room For Debate                  301
S

In [10]:
output_pd = pd.DataFrame(output)
output_pd.columns = ['Category','Predict_count']
rows_count = output_pd.shape[0]
prediction_spread = output_pd['Predict_count'].value_counts()
prediction_spread = pd.DataFrame(prediction_spread)
rows_prediction_count = output_pd.shape[0]
def get_percentage_prediction(num):
    return float(num)/rows_prediction_count
prediction_spread['Percent_predict'] = prediction_spread.Predict_count.apply(get_percentage_prediction)
prediction_spread['Index'] = prediction_spread.index

In [11]:
results = pd.merge(training_spread, prediction_spread, on="Index")

In [12]:
# The report to show data imbalance, but is okay
results = results[['Index','Training_count','Percent_training','Predict_count','Percent_predict']]
results['Percent_predict_more'] = results['Percent_predict'] - results['Percent_training']
results['Percent_predict_more'] = results['Percent_predict_more'].astype('float16')
results

,Index,Training_count,Percent_training,Predict_count,Percent_predict,Percent_predict_more
0,Politics,310682,0.452751,408967,0.557028,1.042480e-01
1,Sunday Review,99848,0.145507,236518,0.322146,1.766357e-01
2,Europe,30698,0.044736,14010,0.019082,-2.565002e-02
3,Middle East,20678,0.030134,9913,0.013502,-1.663208e-02
4,Asia Pacific,17959,0.026171,6220,0.008472,-1.770020e-02
5,Media,12967,0.018897,5939,0.008089,-1.081085e-02
6,Family,10017,0.014598,12208,0.016628,2.029419e-03
7,Television,9343,0.013615,4505,0.006136,-7.480621e-03
8,Economy,7272,0.010597,7701,0.010489,-1.083016e-04
9,Live,6275,0.009144,6560,0.008935,-2.094507e-04


<br>
<br>
<br>

## Save Notebook

In [13]:
from joblib import dump, load
dump(results, 'Models/prediction_results.joblib') 

['Models/prediction_results.joblib']